In [1]:
import gym

### Set Initial Playground for Taxi V3

In [2]:
env = gym.make("Taxi-v3").env
env.render()
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [3]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

### Brute Force without Reinforcement Learning

In [4]:
epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    # for every increase the count by 1 for logging purposes
    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )
    # increase number of epochs
    epochs += 1
    
# Print out the results
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 511
Penalties incurred: 154


In [5]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)

In [6]:
# Simulate Brute Force Approach
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 511
State: 410
Action: 5
Reward: 20


### Using Q-learning reinforcement learning

In [7]:
import numpy as np

In [8]:
# Create a Q learning matrix table by obseravtions X actions
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

#### training the agent

In [9]:
import random
from IPython.display import clear_output

In [10]:
# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

In [11]:
# For plotting metrics
all_epochs = []
all_penalties = []

In [12]:
# Train 100000 for training
for i in range(1, 100001):
    # Reset a new Training simulation
    state = env.reset()
    # set all variables to 0
    epochs, penalties, reward = 0, 0, 0
    done = False
    # Loop unitil simulation is complete
    while not done:
        # If random number between 0 and 1 is less than epsilon
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values
        # retrieve new values
        next_state, reward, done, info = env.step(action)
        # Retrieve old value from Q-table by state and action
        old_value = q_table[state, action]
        
        next_max = np.max(q_table[next_state])
        # use the Q learning equation to determine the new value
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        # Log the new value for given state by action
        q_table[state, action] = new_value
        # log any penalties that occur
        if reward == -10:
            penalties += 1
        # Set the next state
        state = next_state
        # increment the epoch
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")
    
print("Training finished.\n")

Episode: 100000
Training finished.



In [13]:
q_table[328]

array([ -2.40489867,  -2.27325184,  -2.4129062 ,  -2.357825  ,
       -11.16966743, -10.95989298])

### Evaluate agent's performance after Q-learning

In [14]:
reinforcement_frames = [] # for animation

In [15]:
total_epochs, total_penalties = 0, 0
episodes = 100
# Loop through all test episodes
for _ in range(episodes):
    # Start a new test
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    # Run the Q-learning algorithm
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        
        if reward == -10:
            penalties += 1
            
        # Put each rendered frame into dict for animation
        reinforcement_frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward
            }
        )
            
        epochs += 1
        
    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.25
Average penalties per episode: 0.0


In [16]:
print_frames(reinforcement_frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 1325
State: 410
Action: 5
Reward: 20


After training the algorithm with Q-Learning. Reinforcement learning has proven to be much more efficient than blind brute-force as expected. Penalties occured have been reduced to 0 and the average steps have reduced as well.